Get subset of FakeNewCorpus data set

In [ ]:
import pandas as pd
import nltk
from cleantext import clean
data = pd.read_csv('https://raw.githubusercontent.com/several27/FakeNewsCorpus/master/news_sample.csv')
data['content'] = [clean(entry,
                              lower=True,
                              no_line_breaks=True,
                              no_emails=True,
                              no_urls=True,
                              no_numbers=True,
                               lang="en"
                              ) for entry in data['content']]
data['content'].head()

Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.


0    sometimes the power of christmas will make you...
1    awakening of <number> strands of dna "reconnec...
2    never hike alone: a friday the 13th fan film u...
3    when a rare shark was caught, scientists were ...
4    donald trump has the unnerving ability to abil...
Name: content, dtype: object